In [1]:
import numpy as np
from functools import reduce
from sympy import *
from sympy.utilities.lambdify import lambdify

In [5]:
def x4T(x,T): return np.column_stack([x**i for i in range(T.shape[0])]) 
def polynom(x,T): return np.dot(x4T(x,np.array(T)), np.array(T))
def MSE(target): return (polynom(Symbol('x'), target)[0] - Symbol('y_real'))**2/2/(n:= Symbol('n'))
def EMA(x_sum, x, gamma=0.5): return np.array(gamma * np.array(x_sum) + (1 - gamma) * np.array(x))
def RMS(g, grad, alpha=0.01, eta=0.5, et=np.power(10.,-1)): return np.array(alpha*grad) / (np.array(EMA(g, np.array(grad)**2, eta) + et)**0.5)

def GD(func, target_name, arity, n_iter, var, alpha, gamma, eta, **k):
    target_val= np.ones(arity+1)
    var.update({'target': (target:= Array([((locals().update({f'{target_name}{t}': (k:= Symbol(f'{target_name}{t}'))}) is None) & k) for t in range(np.array(target_val).shape[0])])),
                 **{str(t): target_val[i] for i, t in enumerate(target)}})
    func = func(target)
    derr = diff(func, target)
    display(Eq(fx:= (Function(Symbol('f_cost'))(target)), func), Eq(diff(fx, target)[0], derr))
    var_nt_names = list(derr.free_symbols - set(target))
    var_nt_vals = [var[str(i)] for i in var_nt_names]
    derr = Array(map(np.sum, lambdify(var_nt_names, derr)(*var_nt_vals)))
    var_names = Array(list(derr.free_symbols))
    v = derr * 0
    g = 0
    
    for i in range(n_iter):
        sub = [(name, var[str(name)] - gamma * v[i]) for i, name in enumerate(target)]
        target_val = target_val - np.array(v:= EMA(v, np.array(g:= RMS(g, derr.subs(sub), alpha, eta)), gamma))
        target_val_dict ={str(x):target_val[i] for i, x in enumerate(var['target'])}
        var.update(target_val_dict)
        
    display(Eq(Symbol('n_iter'), n_iter), Eq(Symbol('alpha_learn'), alpha), Eq(Symbol('gamma_Nesterov'), gamma), Eq(Symbol('eta_RMSProp'), eta), Eq(target, Array(target_val)))
    return target_val_dict



In [6]:
# Nesterov

params = { 'func': MSE,
           'arity':1,               # prediction polynom arity
           'target_name':'theta', 
           'var': {'x': (X:=np.linspace(-10,10,100)), 
                   'n':  X.shape[0], 
                   'y_real': (Y_real:=polynom(X, (Theta_real:=[2,3])))},
           'n_iter': 100,
           'alpha': 0.04,
           'gamma': 0.5,            # Nesterov koef
           'eta': 0}                # RMSProp koef

thetas = GD(**params)

Eq(f_cost([theta0, theta1]), (theta0 + theta1*x - y_real)**2/(2*n))

Eq(Derivative(f_cost([theta0, theta1]), [theta0, theta1]), [(2*theta0 + 2*theta1*x - 2*y_real)/(2*n), x*(theta0 + theta1*x - y_real)/n])

Eq(n_iter, 100)

Eq(alpha_learn, 0.04)

Eq(gamma_Nesterov, 0.5)

Eq(eta_RMSProp, 0)

Eq([theta0, theta1], [1.99999684016421, 2.99522226927413])

In [7]:
# RMSProp

params = { 'func': MSE,
           'arity':1,               # prediction polynom arity
           'target_name':'theta', 
           'var': {'x': (X:=np.linspace(-10,10,100)), 
                   'n':  X.shape[0], 
                   'y_real': (Y_real:=polynom(X, (Theta_real:=[2,3])))},
           'n_iter': 100,
           'alpha': 0.04,
           'gamma': 0,              # Nesterov koef
           'eta': 0.5}              # RMSProp koef

thetas = GD(**params)

Eq(f_cost([theta0, theta1]), (theta0 + theta1*x - y_real)**2/(2*n))

Eq(Derivative(f_cost([theta0, theta1]), [theta0, theta1]), [(2*theta0 + 2*theta1*x - 2*y_real)/(2*n), x*(theta0 + theta1*x - y_real)/n])

Eq(n_iter, 100)

Eq(alpha_learn, 0.04)

Eq(gamma_Nesterov, 0)

Eq(eta_RMSProp, 0.5)

Eq([theta0, theta1], [1.99999692615498, 3.02189598601617])